In [1]:
import tiktoken
import PyPDF2

In [2]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text


In [13]:
text = extract_text_from_pdf('./data/upload_data/PPIA_45_2024_.pdf')

In [14]:
text

' www.risi.com | 1PPI Asia\nPAPER, PACKAGING AND FIBER MARKET NEWS AND PRICES FOR ASIA\nIn This Issue:\nPrice Watch: China Pulp 2\nPrice Watch: Asia RCP  3\nPrice Watch: China P&B 7\nChina’s Yueyang to buy Hunan Juntai  11\nRengo, Hansol to raise board prices  13China-driven demand bumps prices for US OCC \nimports into Southeast Asia, Taiwan\nMetsä Fibre has extended the maintenance downtime that started on \nNovember 8 at its Kemi pulp and bioproduct mill in northern Finland \nfrom one week to four weeks. But the protracted stoppage will not affect \nthe company’s customer deliveries, for which China is its major market.\nThe company found during the routine maintenance a part of  \nthe recovery boiler at the site needed to be repaired, thus extending  \nthe shut, Metsä Fibre said.\nA company spokesman said that the facility, which has \na capacity of 1.5 million tonnes per year of bleached and \nunbleached softwood and hardwood kraft pulp, has maintained \na higher-than-average leve

In [5]:
def count_tokens(text, model_name):
    # tiktoken supports different encodings for different models
    # For GPT-3.5-turbo or GPT-4, we can use the 'cl100k_base' encoding
    encoding = tiktoken.encoding_for_model(model_name)
    tokens = encoding.encode(text)
    return len(tokens)


In [6]:
count_tokens(text, model_name='gpt-4o')

19026

In [7]:
from langchain_text_splitters import CharacterTextSplitter

In [27]:
splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len
)

In [28]:
splitted_text = splitter.create_documents([text])

In [29]:
print(splitted_text[0])

page_content='www.risi.com | 1PPI Asia
PAPER, PACKAGING AND FIBER MARKET NEWS AND PRICES FOR ASIA
In This Issue:
Price Watch: China Pulp 2
Price Watch: Asia RCP  3
Price Watch: China P&B 7
China’s Yueyang to buy Hunan Juntai  11
Rengo, Hansol to raise board prices  13China-driven demand bumps prices for US OCC 
imports into Southeast Asia, Taiwan
Metsä Fibre has extended the maintenance downtime that started on 
November 8 at its Kemi pulp and bioproduct mill in northern Finland 
from one week to four weeks. But the protracted stoppage will not affect 
the company’s customer deliveries, for which China is its major market.
The company found during the routine maintenance a part of  
the recovery boiler at the site needed to be repaired, thus extending  
the shut, Metsä Fibre said.
A company spokesman said that the facility, which has 
a capacity of 1.5 million tonnes per year of bleached and 
unbleached softwood and hardwood kraft pulp, has maintained 
a higher-than-average level of st

In [30]:
test = splitted_text[0]

In [34]:
len(splitted_text[0].page_content)

69683